In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TPU node sandbox



## Environment Setup

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

import os
import tensorflow_datasets as tfds

import tensorflow_hub as hub
import tensorflow_text as text

from cloud_tpu_client import Client

2021-09-10 16:12:41.795603: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-10 16:12:41.795645: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Configure GCP settings

In [2]:
PROJECT = 'jk-mlops-dev'
REGION = 'us-central1'
TPU_NODE_NAME = 'jk-tpu-node'
ZONE = 'us-central1-a'
GCS_BUCKET = 'gs://jk-tpu-staging'

c = Client(tpu=TPU_NODE_NAME, zone=ZONE)

print(c.accelerator_type())
print(c.name())
print(c.state())
print(c.runtime_version())

v3-8
jk-tpu-node
READY
2.5.0


In [3]:
c.configure_tpu_version(tf.__version__, restart_type='ifNeeded')
c.wait_for_healthy()

Note: The TPU initialization code has to be at the beginning of your program.

In [4]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_NODE_NAME)
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: jk-tpu-node


2021-09-10 16:12:50.092418: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-10 16:12:50.092467: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-10 16:12:50.092490: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jk-tpu-node-1): /proc/driver/nvidia/version does not exist
2021-09-10 16:12:50.253208: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-10 16:12:50.266667: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcCha

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [5]:
MODEL_URI = 'gs://tfhub-modules/tensorflow/bert_en_cased_L-24_H-1024_A-16/3/uncompressed'
PREPROCESSOR_URI = 'gs://tfhub-modules/tensorflow/bert_en_cased_preprocess/3/uncompressed'

In [6]:
def get_bert_model(seq_len=128):
    
    bert_layer = hub.KerasLayer(
        handle=MODEL_URI, 
        trainable=True,
        name='encoder'
    )
    encoder_inputs = dict(
        input_word_ids = tf.keras.layers.Input(shape=(seq_len,), dtype=tf.int32, name="input_word_ids"),
        input_mask = tf.keras.layers.Input(shape=(seq_len,), dtype=tf.int32, name="input_mask"),
        input_type_ids = tf.keras.layers.Input(shape=(seq_len,), dtype=tf.int32, name="segment_ids"),
    )
    encoder_outputs = bert_layer(encoder_inputs)
    embedding = encoder_outputs["pooled_output"]
    logits = tf.keras.layers.Dense(1)(embedding)
    softmax_prob = tf.keras.layers.Softmax()(logits)
    model = tf.keras.models.Model(inputs=encoder_inputs,
                                  outputs=softmax_prob)
    
    return model

In [7]:
def create_bert_preprocess_model(seq_length=128):

  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')

  bert_preprocess = hub.load(PREPROCESSOR_URI)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')                                  
                                     
  tokenized_inputs = [tokenizer(segment) for segment in [text_input]]
  model_inputs = packer(tokenized_inputs)
                                     
  return tf.keras.Model(text_input, model_inputs)

In [8]:
tx = tf.constant(['abc def','xyz abc apple', 'abc xyz', 'appel pen'], dtype=tf.string)
ty = tf.constant([1, 1, 0, 0], dtype=tf.int64)
vx = tf.constant(['apple peer','abc apple', 'xyz', 'vvvv'], dtype=tf.string)
vy = tf.constant([1, 0, 0, 1], dtype=tf.int64)

In [9]:
preprocessor = create_bert_preprocess_model()

In [10]:
tx_inputs = preprocessor(tx)

In [11]:
strategy = tf.distribute.TPUStrategy(resolver)
#strategy = tf.distribute.get_strategy()

with strategy.scope():
    model = get_bert_model()
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
    model.summary()

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
encoder (KerasLayer)            {'pooled_output': (N 333579265   input_mask[0][0]                 
                                                                 segment_ids[0][0]          

In [12]:
model.fit(
    x=tx_inputs,
    y=ty,
    epochs=10
)

Epoch 1/10


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/StatefulPartitionedCall:1", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/StatefulPartitionedCall:0", dtype=float32), dense_shape=Tensor("Adam/gradients/StatefulPartitionedCall:2", shape=(None,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


1/1 [==============================] - 132s 132s/step - loss: 7.6666 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 112ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 111ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 112ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 110ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 111ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 111ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 8/10
1/1 [==============================] - 0s 109ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 9/10
1/1 [==============================] - 0s 111ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 10/10
1/1 [==============================] - 0s 111ms/step - loss: 7.6666 - accuracy: 0.5000


In [19]:
#dataset = tf.data.Dataset.from_tensor_slices((tx_inputs, ty)).batch(2)

dataset = tf.data.Dataset.from_tensor_slices((tx, ty)).batch(2)
dataset = dataset.map(lambda x, y: (preprocessor(x), y))


In [20]:
model.fit(x=dataset,
          epochs=10)

Epoch 1/10
2/2 [==============================] - 0s 113ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 0s 114ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 0s 111ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 0s 113ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 0s 113ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 6/10
2/2 [==============================] - 0s 113ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 0s 113ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 8/10
2/2 [==============================] - 0s 112ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 9/10
2/2 [==============================] - 0s 119ms/step - loss: 7.6666 - accuracy: 0.5000
Epoch 10/10
2/2 [==============================] - 0s 113ms/step - loss: 7.6666 - accuracy: 0.5000
